# Toronto neighborhoods - Part I

1. Step - Import Libraries Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

In [23]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [27]:
# download data from url given in the task
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
Canada_data = BeautifulSoup(source, 'lxml')

In [25]:
# creat a new Dataframe
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

In [28]:
content = Canada_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

# clean dataframe 
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
                                 
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,","
1,M1B\n,Scarborough\n,"Malvern / Rouge, Malvern / Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill / Port Union / Highland Creek, Roug..."
3,M1E\n,Scarborough\n,"Guildwood / Morningside / West Hill, Guildwood..."
4,M1G\n,Scarborough\n,"Woburn, Woburn"


In [35]:
# Get names of indexes for which column Borough has value "Not assigned"
indexNames = df[ df['Borough'] =='Not assigned'].index

# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)

In [36]:
df.head(10)

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,","
1,M1B\n,Scarborough\n,"Malvern / Rouge, Malvern / Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill / Port Union / Highland Creek, Roug..."
3,M1E\n,Scarborough\n,"Guildwood / Morningside / West Hill, Guildwood..."
4,M1G\n,Scarborough\n,"Woburn, Woburn"
5,M1H\n,Scarborough\n,"Cedarbrae, Cedarbrae"
6,M1J\n,Scarborough\n,"Scarborough Village, Scarborough Village"
7,M1K\n,Scarborough\n,"Kennedy Park / Ionview / East Birchmount Park,..."
8,M1L\n,Scarborough\n,"Golden Mile / Clairlea / Oakridge, Golden Mile..."
9,M1M\n,Scarborough\n,Cliffside / Cliffcrest / Scarborough Village W...


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [40]:
df.loc[df['Borough'] =='Not assigned' , 'Neighborhood'] = df['Borough']
df.head(10)

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,","
1,M1B\n,Scarborough\n,"Malvern / Rouge, Malvern / Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill / Port Union / Highland Creek, Roug..."
3,M1E\n,Scarborough\n,"Guildwood / Morningside / West Hill, Guildwood..."
4,M1G\n,Scarborough\n,"Woburn, Woburn"
5,M1H\n,Scarborough\n,"Cedarbrae, Cedarbrae"
6,M1J\n,Scarborough\n,"Scarborough Village, Scarborough Village"
7,M1K\n,Scarborough\n,"Kennedy Park / Ionview / East Birchmount Park,..."
8,M1L\n,Scarborough\n,"Golden Mile / Clairlea / Oakridge, Golden Mile..."
9,M1M\n,Scarborough\n,Cliffside / Cliffcrest / Scarborough Village W...


In [41]:
result = df.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)

In [43]:
df_new=result.reset_index()
df_new.head(15)

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,","
1,M1B\n,Scarborough\n,"Malvern / Rouge, Malvern / Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill / Port Union / Highland Creek, Roug..."
3,M1E\n,Scarborough\n,"Guildwood / Morningside / West Hill, Guildwood..."
4,M1G\n,Scarborough\n,"Woburn, Woburn"
5,M1H\n,Scarborough\n,"Cedarbrae, Cedarbrae"
6,M1J\n,Scarborough\n,"Scarborough Village, Scarborough Village"
7,M1K\n,Scarborough\n,"Kennedy Park / Ionview / East Birchmount Park,..."
8,M1L\n,Scarborough\n,"Golden Mile / Clairlea / Oakridge, Golden Mile..."
9,M1M\n,Scarborough\n,Cliffside / Cliffcrest / Scarborough Village W...


In [44]:
df_new.shape

(180, 3)